# CNCF Landscape: {Category, Subcategory, Project } --> Stars, Commits, Contributors, Activity

### On JupyterLab Shell Integration(s) and SList 

More Info: _http://safaribooksonline.com/blog/2014/02/12/using-shell-commands-effectively-ipython_

SList instances can be used like a regular list, but they provide several methods that are useful when working with shell output. The main properties available in an SList instance are:



* `.s` returns the elements joined together by spaces. 
  * _This is useful for building command lines that take many arguments in a single invocation._
* `.n` returns the elements joined together by a newline. 
  * _Use this when you need the original output unmodified._
* `.p` returns the elements as path objects, if they are filenames.\
  * _Use this when doing more advanced path manipulation_

In addition, SList instances support `grep()` and `fields()` methods. 


In [104]:
!pip list

Package                            Version
---------------------------------- ------------
aiohttp                            3.8.6
aiosignal                          1.3.1
aiosqlite                          0.19.0
altair                             5.1.2
annotated-types                    0.6.0
anyio                              3.7.1
appnope                            0.1.3
argon2-cffi                        23.1.0
argon2-cffi-bindings               21.2.0
arrow                              1.3.0
astroid                            3.0.1
asttokens                          2.4.1
async-lru                          2.0.4
async-timeout                      4.0.3
attrs                              23.1.0
autopep8                           2.0.4
Babel                              2.13.1
backoff                            2.2.1
beautifulsoup4                     4.12.2
bleach                             6.1.0
bokeh                              3.3.1
botocore                           1.32.4


In [105]:
%load_ext jupyter_ai_magics

The jupyter_ai_magics extension is already loaded. To reload it, use:
  %reload_ext jupyter_ai_magics


In [106]:
%ai list

| Provider | Environment variable | Set? | Models |
|----------|----------------------|------|--------|
| `ai21` | `AI21_API_KEY` | <abbr title="You have not set this environment variable, so you cannot use this provider's models.">❌</abbr> | `ai21:j1-large`, `ai21:j1-grande`, `ai21:j1-jumbo`, `ai21:j1-grande-instruct`, `ai21:j2-large`, `ai21:j2-grande`, `ai21:j2-jumbo`, `ai21:j2-grande-instruct`, `ai21:j2-jumbo-instruct` |
| `bedrock` | Not applicable. | <abbr title="Not applicable">N/A</abbr> | `bedrock:amazon.titan-text-express-v1`, `bedrock:ai21.j2-ultra-v1`, `bedrock:ai21.j2-mid-v1`, `bedrock:cohere.command-text-v14` |
| `bedrock-chat` | Not applicable. | <abbr title="Not applicable">N/A</abbr> | `bedrock-chat:anthropic.claude-v1`, `bedrock-chat:anthropic.claude-v2`, `bedrock-chat:anthropic.claude-instant-v1` |
| `anthropic` | `ANTHROPIC_API_KEY` | <abbr title="You have not set this environment variable, so you cannot use this provider's models.">❌</abbr> | `anthropic:claude-v1`, `anthropic:claude-v1.0`, `anthropic:claude-v1.2`, `anthropic:claude-2`, `anthropic:claude-2.0`, `anthropic:claude-instant-v1`, `anthropic:claude-instant-v1.0`, `anthropic:claude-instant-v1.2` |
| `anthropic-chat` | `ANTHROPIC_API_KEY` | <abbr title="You have not set this environment variable, so you cannot use this provider's models.">❌</abbr> | `anthropic-chat:claude-v1`, `anthropic-chat:claude-v1.0`, `anthropic-chat:claude-v1.2`, `anthropic-chat:claude-2`, `anthropic-chat:claude-2.0`, `anthropic-chat:claude-instant-v1`, `anthropic-chat:claude-instant-v1.0`, `anthropic-chat:claude-instant-v1.2` |
| `azure-chat-openai` | `OPENAI_API_KEY` | <abbr title="You have not set this environment variable, so you cannot use this provider's models.">❌</abbr> | This provider does not define a list of models. |
| `cohere` | `COHERE_API_KEY` | <abbr title="You have not set this environment variable, so you cannot use this provider's models.">❌</abbr> | `cohere:medium`, `cohere:xlarge` |
| `gpt4all` | Not applicable. | <abbr title="Not applicable">N/A</abbr> | `gpt4all:ggml-gpt4all-j-v1.2-jazzy`, `gpt4all:ggml-gpt4all-j-v1.3-groovy`, `gpt4all:ggml-gpt4all-l13b-snoozy`, `gpt4all:mistral-7b-openorca.Q4_0`, `gpt4all:mistral-7b-instruct-v0.1.Q4_0`, `gpt4all:gpt4all-falcon-q4_0`, `gpt4all:wizardlm-13b-v1.2.Q4_0`, `gpt4all:nous-hermes-llama2-13b.Q4_0`, `gpt4all:gpt4all-13b-snoozy-q4_0`, `gpt4all:mpt-7b-chat-merges-q4_0`, `gpt4all:orca-mini-3b-gguf2-q4_0`, `gpt4all:starcoder-q4_0`, `gpt4all:rift-coder-v0-7b-q4_0`, `gpt4all:em_german_mistral_v01.Q4_0` |
| `huggingface_hub` | `HUGGINGFACEHUB_API_TOKEN` | <abbr title="You have not set this environment variable, so you cannot use this provider's models.">❌</abbr> | See [https://huggingface.co/models](https://huggingface.co/models) for a list of models. Pass a model's repository ID as the model ID; for example, `huggingface_hub:ExampleOwner/example-model`. |
| `openai` | `OPENAI_API_KEY` | <abbr title="You have not set this environment variable, so you cannot use this provider's models.">❌</abbr> | `openai:text-davinci-003`, `openai:text-davinci-002`, `openai:text-curie-001`, `openai:text-babbage-001`, `openai:text-ada-001`, `openai:davinci`, `openai:curie`, `openai:babbage`, `openai:ada` |
| `openai-chat` | `OPENAI_API_KEY` | <abbr title="You have not set this environment variable, so you cannot use this provider's models.">❌</abbr> | `openai-chat:gpt-3.5-turbo`, `openai-chat:gpt-3.5-turbo-16k`, `openai-chat:gpt-3.5-turbo-0301`, `openai-chat:gpt-3.5-turbo-0613`, `openai-chat:gpt-3.5-turbo-16k-0613`, `openai-chat:gpt-4`, `openai-chat:gpt-4-0314`, `openai-chat:gpt-4-0613`, `openai-chat:gpt-4-32k`, `openai-chat:gpt-4-32k-0314`, `openai-chat:gpt-4-32k-0613` |
| `openai-chat-new` | `OPENAI_API_KEY` | <abbr title="You have not set this environment variable, so you cannot use this provider's models.">❌</abbr> | `openai-chat-new:gpt-3.5-turbo`, `openai-chat-new:gpt-3.5-turbo-16k`, `openai-chat-new:gpt-3.5-turbo-0301`, `openai-chat-new:gpt-3.5-turbo-0613`, `openai-chat-new:gpt-3.5-turbo-16k-0613`, `openai-chat-new:gpt-4`, `openai-chat-new:gpt-4-0314`, `openai-chat-new:gpt-4-0613`, `openai-chat-new:gpt-4-32k`, `openai-chat-new:gpt-4-32k-0314`, `openai-chat-new:gpt-4-32k-0613` |
| `sagemaker-endpoint` | Not applicable. | <abbr title="Not applicable">N/A</abbr> | Specify an endpoint name as the model ID. In addition, you must specify a region name, request schema, and response path. For more information, see the documentation about [SageMaker endpoints deployment](https://docs.aws.amazon.com/sagemaker/latest/dg/realtime-endpoints-deployment.html) and about [using magic commands with SageMaker endpoints](https://jupyter-ai.readthedocs.io/en/latest/users/index.html#using-magic-commands-with-sagemaker-endpoints). |

Aliases and custom commands:

| Name | Target |
|------|--------|
| `gpt2` | `huggingface_hub:gpt2` |
| `gpt3` | `openai:text-davinci-003` |
| `chatgpt` | `openai-chat:gpt-3.5-turbo` |
| `gpt4` | `openai-chat:gpt-4` |
| `titan` | `bedrock:amazon.titan-tg1-large` |


In [107]:
%ai help

Usage: %%ai [OPTIONS] MODEL_ID

  Invokes a language model identified by MODEL_ID, with the prompt being
  contained in all lines after the first. Both local model IDs and global
  model IDs (with the provider ID explicitly prefixed, followed by a colon)
  are accepted.

  To view available language models, please run `%ai list`.

Options:
  -f, --format [code|html|image|json|markdown|math|md|text]
                                  IPython display to use when rendering
                                  output. [default="markdown"]
  -r, --reset                     Clears the conversation transcript used when
                                  interacting with an OpenAI chat model
                                  provider. Does nothing with other providers.
  -n, --region-name TEXT          AWS region name, e.g. 'us-east-1'. Required
                                  for SageMaker provider; does nothing with
                                  other providers.
  -q, --request-schema TEXT 

## Base imports and variables

In [108]:
import os
import pandas as pd

import sys

from dotenv import load_dotenv

load_dotenv()

OUT_DIR='generated'
CNCF_LANDSCAPE_FNAME_BASE='cncf-landscape'
CNCF_LANDSCAPE_FNAME_ROOT=f'{OUT_DIR}/{CNCF_LANDSCAPE_FNAME_BASE}'

CNCF_PROJECTS_FNAME_BASE=f'cncf-projects'
CNCF_PROJECTS_FNAME_ROOT=f'{OUT_DIR}/{CNCF_PROJECTS_FNAME_BASE}'

print(f'Jupyter Kernel (venv): {sys.executable}')
print(f'Output Location:       {OUT_DIR}  (.json, .jsonl, .csv, .md, ...)')
print(f'Output Landscape root: {CNCF_LANDSCAPE_FNAME_ROOT}-*')
print(f'Output Projects  root: {CNCF_PROJECTS_FNAME_ROOT}-*')

Jupyter Kernel (venv): /Users/matt/gh/cncf/landscape-graph/.venv-ipynb/bin/python
Output Location:       generated  (.json, .jsonl, .csv, .md, ...)
Output Landscape root: generated/cncf-landscape-*
Output Projects  root: generated/cncf-projects-*


## download landscape data

In [109]:
# create directory for generated files
!mkdir -p {OUT_DIR}

#Get the current landscape json (array) file
!wget -O {CNCF_LANDSCAPE_FNAME_ROOT}.json.compact https://landscape.cncf.io/data/items.json
!ls -lh {CNCF_LANDSCAPE_FNAME_ROOT}.json.compact

--2023-11-27 12:51:53--  https://landscape.cncf.io/data/items.json
Resolving landscape.cncf.io (landscape.cncf.io)... 2600:1f18:16e:df00::64, 2600:1f18:2489:8200::c8, 44.217.161.11, ...
Connecting to landscape.cncf.io (landscape.cncf.io)|2600:1f18:16e:df00::64|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10200329 (9.7M) [application/json]
Saving to: ‘generated/cncf-landscape.json.compact’

generated/cncf-land 100%[===================>]   9.73M  5.78MB/s    in 1.7s    

2023-11-27 12:51:55 (5.78 MB/s) - ‘generated/cncf-landscape.json.compact’ saved [10200329/10200329]

-rw-r--r--  1 matt  staff   9.7M Nov 27 12:51 generated/cncf-landscape.json.compact


### compact -> human friendly, generate csv

In [110]:
!jq . {CNCF_LANDSCAPE_FNAME_ROOT}.json.compact > {CNCF_LANDSCAPE_FNAME_ROOT}.json

!echo "Yes indeed, that's 2+ MB of whitespace!\n"
!ls -lh {CNCF_LANDSCAPE_FNAME_ROOT}.json* 

Yes indeed, that's 2+ MB of whitespace!

-rw-r--r--  1 matt  staff    12M Nov 27 12:51 generated/cncf-landscape.json
-rw-r--r--  1 matt  staff   9.7M Nov 27 12:51 generated/cncf-landscape.json.compact
-rw-r--r--  1 matt  staff   9.7M Nov 27 12:50 generated/cncf-landscape.jsonl


In [111]:
# !jq '.items[] | [.relation, .category, .subcategory, .id, .name, .flatName, .repo, .repo_name, .org_name, .contributorsCount, .commitsThisYear, .stars, .github_data, .extra, .industries, .headquarters, .crunchbase, .image_data .twitter] | @csv'  {CNCF_LANDSCAPE_FNAME_ROOT}.json.compact >  {CNCF_LANDSCAPE_FNAME_ROOT}.csv
# !ls -lh  {CNCF_LANDSCAPE_FNAME_ROOT}.csv

In [112]:
# array of JSON --> JSONL
!jq  -c '.[]'  {CNCF_LANDSCAPE_FNAME_ROOT}.json >  {CNCF_LANDSCAPE_FNAME_ROOT}.jsonl

## Filter 1200+ cards/entities --> ~180 CNCF Projects --> cncf-projects.jsonl

In [113]:
%%bash

ls -lahF {CNCF_LANDSCAPE_FNAME_ROOT}.jsonl
wc -l    {CNCF_LANDSCAPE_FNAME_ROOT}.jsonl

# landscape | select(CNCF Projects) --> cncf-projects.jsonl
jq -c 'select(.relation == "graduated" or .relation == "incubating" or .relation == "sandbox")'  {CNCF_LANDSCAPE_FNAME_ROOT}.jsonl > cncf-projects.jsonl 

ls -lahF cncf-projects.jsonl
wc -l cncf-projects.jsonl

-rw-r--r--  1 matt  staff     0B Nov 22 15:41 {CNCF_LANDSCAPE_FNAME_ROOT}.jsonl
       0 {CNCF_LANDSCAPE_FNAME_ROOT}.jsonl
-rw-r--r--  1 matt  staff     0B Nov 27 12:51 cncf-projects.jsonl
       0 cncf-projects.jsonl


### DataFrame helpers (safe_set_index(), split_org_repo()

In [114]:
import pandas as pd

pd.set_option('display.max_rows', 512)
pd.set_option('display.max_columns', 512)
pd.set_option('display.width', 512)

# https://plotly.com/python/pandas-backend
pd.options.plotting.backend = "plotly"

def safe_set_index(df:         pd.DataFrame, 
                   idx_wanted: list[str],
                   sort:       bool = True,
                   inplace:    bool = True) -> pd.DataFrame:
    '''check to see if the index is already set, else, data loss as set_index can be destructive'''
    
    idx_existing = list(df.index.names)

    if idx_wanted == idx_existing:
        print(f'\n*** WARNING: attempt to set index to what it already is thwarted! \n')
    else:
        df.set_index(idx_wanted, verify_integrity=True, inplace=inplace)
        print(f'\t Index changed from {idx_existing} --> {list(df.index.names)}') 

    if sort:
        df.sort_index(inplace=inplace)

    return df

def split_org_repo(df:      pd.DataFrame, 
                   colname: str,
                   drop:    bool = False,
                   newcol_org_name:  str = 'org_name',
                   newcol_repo_name: str = 'repo_name') -> pd.DataFrame:
    '''split_org_repo(df, colname) - org_name/repo_name --> org_name, repo_name'''
    
    if colname is None:
        raise ValueError('split_org_repo: missing colname!')

    # https://swdevnotes.com/python/2022/extract-data-from-json-in-pandas-dataframe/
    # expand=True returns a dataframe  which we can rename columns on
    
    df_newcols = df[colname].copy().str.split(pat='/', n=1, expand=True)
    df_newcols.rename(columns={0: newcol_org_name, 1: newcol_repo_name}, inplace=True)

    if drop:
        df.drop(colname, axis=1, inplace=True)

    df = pd.concat([df,df_newcols], axis=1)
    return df

## Load and prep the Landscape from cncf-projects.jsonl

In [115]:
file_path = f'{CNCF_LANDSCAPE_FNAME_ROOT}.jsonl'

if os.path.exists(file_path) and os.path.getsize(file_path) > 0:
    df_projects = pd.read_json(file_path, lines=True)
else:
    print(f"File {file_path} does not exist or is empty.")

print(df_projects.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2276 entries, 0 to 2275
Data columns (total 60 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   name                      2276 non-null   object 
 1   homepage_url              2276 non-null   object 
 2   repo_url                  755 non-null    object 
 3   logo                      2276 non-null   object 
 4   twitter                   1882 non-null   object 
 5   crunchbase                2274 non-null   object 
 6   github_data               755 non-null    object 
 7   repos                     755 non-null    object 
 8   github_start_commit_data  755 non-null    object 
 9   image_data                2276 non-null   object 
 10  firstCommitDate           755 non-null    object 
 11  firstCommitLink           755 non-null    object 
 12  latestCommitDate          755 non-null    object 
 13  latestCommitLink          755 non-null    object 
 14  releaseD

In [116]:
df_projects.head()

,name,homepage_url,repo_url,logo,twitter,crunchbase,github_data,repos,github_start_commit_data,image_data,firstCommitDate,firstCommitLink,latestCommitDate,latestCommitLink,releaseDate,releaseLink,commitsThisYear,contributorsCount,contributorsLink,language,stars,license,headquarters,latestTweetDate,description,organization,crunchbaseData,path,landscape,category,amount,oss,href,bestPracticeBadgeId,bestPracticePercentage,industries,starsPresent,starsAsText,marketCapPresent,marketCapAsText,id,flatName,member,relation,isSubsidiaryProject,project,extra,amountKind,yahoo_finance_data,allow_duplicate_repo,joined,url_for_bestpractices,project_org,open_source,enduser,stock_ticker,second_path,branch,hideLicense,unnamed_organization
0,Airship,https://www.airshipit.org/,https://github.com/airshipit/treasuremap,airship.svg,https://twitter.com/airshipproject,https://www.crunchbase.com/organization/open-i...,"{'languages': [{'name': 'Shell', 'value': 3022...",[{'url': 'https://github.com/airshipit/treasur...,{'start_commit_link': '/airshipit/treasuremap/...,"{'fileName': 'airship.svg', 'hash': 'kS96vnkPA...",2018-08-14T23:50:18Z,https://github.com/airshipit/treasuremap/commi...,2021-11-18T19:35:10Z,https://github.com/airshipit/treasuremap/commi...,2021-12-06T20:04:17Z,https://github.com/airshipit/treasuremap/releases,0,117.0,https://github.com/airshipit/treasuremap/graph...,Shell,49.0,Apache License 2.0,"Austin, Texas","{'text': '8 months ago', 'value': '890', 'orig...","Reference Airship manifests, CICD, and referen...",Open Infrastructure Foundation,"{'name': 'Open Infrastructure Foundation', 'de...",Provisioning / Automation & Configuration,Provisioning / Automation & Configuration,Provisioning,N/A,True,logos/airship.svg,0,NaN,[],True,49,False,N/A,airship,Airship,False,False,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Akri,https://docs.akri.sh,https://github.com/project-akri/akri,akri.svg,https://twitter.com/ProjectAkri,https://www.crunchbase.com/organization/cloud-...,"{'languages': [{'name': 'Rust', 'value': 10535...",[{'url': 'https://github.com/project-akri/akri...,{'start_commit_link': '/project-akri/akri/comm...,"{'fileName': 'akri.svg', 'hash': '2nK42JQaM8qF...",2020-10-14T00:42:19Z,https://github.com/project-akri/akri/commit/94...,"{'text': '3 months ago', 'value': '940', 'orig...",https://github.com/project-akri/akri/commit/98...,"{'text': '7 months ago', 'value': '900', 'orig...",https://github.com/project-akri/akri/releases,197,34.0,https://github.com/project-akri/akri/graphs/co...,Rust,996.0,Apache License 2.0,"San Francisco, California","{'text': '7 months ago', 'value': '900', 'orig...",A Kubernetes Resource Interface for the Edge,Cloud Native Computing Foundation (CNCF),{'name': 'Cloud Native Computing Foundation (C...,Provisioning / Automation & Configuration,Provisioning / Automation & Configuration,Provisioning,3000000,True,logos/akri.svg,5339,99.0,"[Cloud Computing, Cloud Infrastructure, Non Pr...",True,996,True,$3M,akri,Akri,False,sandbox,False,sandbox,"{'accepted': '2021-09-14', 'annual_review_date...",funding,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Ansible,https://www.ansible.com/,https://github.com/ansible/ansible,ansible.svg,https://twitter.com/ansible,https://www.crunchbase.com/organization/red-hat,"{'languages': [{'name': 'Python', 'value': 927...","[{'url': 'https://github.com/ansible/ansible',...",{'start_commit_link': '/ansible/ansible/commit...,"{'fileName': 'ansible.svg', 'hash': 'XHYuAuMuT...",2012-02-23T19:17:24Z,https://github.com/ansible/ansible/commit/f314...,"{'text': '2 months ago', 'value': '950', 'orig...",https://github.com/ansible/ansible/commit/7d98...,"{'text': '3 months ago', 'value': '940', 'orig...",https://github.com/ansible/ansible/releases,959,6780.0,https://github.com/ansible/ansible/graphs/cont...,Python,58659.0,GNU General Public License v3.0,"Raleigh, North Carolina","{'text': '7 months ago', 'value': '900', 'orig...",Ansible is a radically simple IT aut

### cols += { subcategory, repo, org_name, repo_name }

In [117]:
# clean up path to strip out the category (path := category / subcategory)
df_projects['subcategory'] = df_projects['path'].str.split('/').str[-1]

# https://github.com/theOrg/theRepo --> theOrg/theRepo
df_projects['repo'] = df_projects['repo_url'].astype('string').str.removeprefix('https://github.com/')

# theOrg/theRepo --> theOrg, theRepo
df_projects = split_org_repo(df_projects, 'repo')

In [118]:
df_projects.head(3)

,name,homepage_url,repo_url,logo,twitter,crunchbase,github_data,repos,github_start_commit_data,image_data,firstCommitDate,firstCommitLink,latestCommitDate,latestCommitLink,releaseDate,releaseLink,commitsThisYear,contributorsCount,contributorsLink,language,stars,license,headquarters,latestTweetDate,description,organization,crunchbaseData,path,landscape,category,amount,oss,href,bestPracticeBadgeId,bestPracticePercentage,industries,starsPresent,starsAsText,marketCapPresent,marketCapAsText,id,flatName,member,relation,isSubsidiaryProject,project,extra,amountKind,yahoo_finance_data,allow_duplicate_repo,joined,url_for_bestpractices,project_org,open_source,enduser,stock_ticker,second_path,branch,hideLicense,unnamed_organization,subcategory,repo,org_name,repo_name
0,Airship,https://www.airshipit.org/,https://github.com/airshipit/treasuremap,airship.svg,https://twitter.com/airshipproject,https://www.crunchbase.com/organization/open-i...,"{'languages': [{'name': 'Shell', 'value': 3022...",[{'url': 'https://github.com/airshipit/treasur...,{'start_commit_link': '/airshipit/treasuremap/...,"{'fileName': 'airship.svg', 'hash': 'kS96vnkPA...",2018-08-14T23:50:18Z,https://github.com/airshipit/treasuremap/commi...,2021-11-18T19:35:10Z,https://github.com/airshipit/treasuremap/commi...,2021-12-06T20:04:17Z,https://github.com/airshipit/treasuremap/releases,0,117.0,https://github.com/airshipit/treasuremap/graph...,Shell,49.0,Apache License 2.0,"Austin, Texas","{'text': '8 months ago', 'value': '890', 'orig...","Reference Airship manifests, CICD, and referen...",Open Infrastructure Foundation,"{'name': 'Open Infrastructure Foundation', 'de...",Provisioning / Automation & Configuration,Provisioning / Automation & Configuration,Provisioning,N/A,True,logos/airship.svg,0,NaN,[],True,49,False,N/A,airship,Airship,False,False,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Automation & Configuration,airshipit/treasuremap,airshipit,treasuremap
1,Akri,https://docs.akri.sh,https://github.com/project-akri/akri,akri.svg,https://twitter.com/ProjectAkri,https://www.crunchbase.com/organization/cloud-...,"{'languages': [{'name': 'Rust', 'value': 10535...",[{'url': 'https://github.com/project-akri/akri...,{'start_commit_link': '/project-akri/akri/comm...,"{'fileName': 'akri.svg', 'hash': '2nK42JQaM8qF...",2020-10-14T00:42:19Z,https://github.com/project-akri/akri/commit/94...,"{'text': '3 months ago', 'value': '940', 'orig...",https://github.com/project-akri/akri/commit/98...,"{'text': '7 months ago', 'value': '900', 'orig...",https://github.com/project-akri/akri/releases,197,34.0,https://github.com/project-akri/akri/graphs/co...,Rust,996.0,Apache License 2.0,"San Francisco, California","{'text': '7 months ago', 'value': '900', 'orig...",A Kubernetes Resource Interface for the Edge,Cloud Native Computing Foundation (CNCF),{'name': 'Cloud Native Computing Foundation (C...,Provisioning / Automation & Configuration,Provisioning / Automation & Configuration,Provisioning,3000000,True,logos/akri.svg,5339,99.0,"[Cloud Computing, Cloud Infrastructure, Non Pr...",True,996,True,$3M,akri,Akri,False,sandbox,False,sandbox,"{'accepted': '2021-09-14', 'annual_review_date...",funding,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Automation & Configuration,project-akri/akri,project-akri,akri
2,Ansible,https://www.ansible.com/,https://github.com/ansible/ansible,ansible.svg,https://twitter.com/ansible,https://www.crunchbase.com/organization/red-hat,"{'languages': [{'name': 'Python', 'value': 927...","[{'url': 'https://github.com/ansible/ansible',...",{'start_commit_link': '/ansible/ansible/commit...,"{'fileName': 'ansible.svg', 'hash': 'XHYuAuMuT...",2012-02-23T19:17:24Z,https://github.com/ansible/ansible/commit/f314...,"{'text': '2 months ago', 'value': '950', 'orig...",https://github.com/ansible/ansible/commit/7d98...,"{'text': '3 months ago', 'value': '940', 'orig...",https://github.com/ansible/ansible/releases,959,6780.0,https://github.com/ansible/ansible/graphs

## Create a smaller (cleaner) dataframe to work with

In [119]:
# df2=df[['B','D','F']].rename({'B':'X','D':'Y','F':'Z'}, axis=1)
df = df_projects.reset_index()
df = df[[
    'relation',
    'category',
    'subcategory',
    'id',
    'name',
    'flatName',
    'repo',
    'repo_name',
    'org_name',
    'contributorsCount',
    'commitsThisYear',
    'stars',
    'github_data',
    'extra',
    'industries',
    'headquarters',
    'image_data']].copy()

df.head()

,relation,category,subcategory,id,name,flatName,repo,repo_name,org_name,contributorsCount,commitsThisYear,stars,github_data,extra,industries,headquarters,image_data
0,False,Provisioning,Automation & Configuration,airship,Airship,Airship,airshipit/treasuremap,treasuremap,airshipit,117.0,0,49.0,"{'languages': [{'name': 'Shell', 'value': 3022...",NaN,[],"Austin, Texas","{'fileName': 'airship.svg', 'hash': 'kS96vnkPA..."
1,sandbox,Provisioning,Automation & Configuration,akri,Akri,Akri,project-akri/akri,akri,project-akri,34.0,197,996.0,"{'languages': [{'name': 'Rust', 'value': 10535...","{'accepted': '2021-09-14', 'annual_review_date...","[Cloud Computing, Cloud Infrastructure, Non Pr...","San Francisco, California","{'fileName': 'akri.svg', 'hash': '2nK42JQaM8qF..."
2,member,Provisioning,Automation & Configuration,ansible,Ansible,Ansible,ansible/ansible,ansible,ansible,6780.0,959,58659.0,"{'languages': [{'name': 'Python', 'value': 927...",NaN,"[Enterprise Software, InsurTech, Linux, Open S...","Raleigh, North Carolina","{'fileName': 'ansible.svg', 'hash': 'XHYuAuMuT..."
3,False,Provisioning,Automation & Configuration,apollo,Apollo,Apollo,apolloconfig/apollo,apollo,apolloconfig,170.0,96,28360.0,"{'languages': [{'name': 'Java', 'value': 22675...",NaN,"[Hospitality, Tourism, Travel]","Singapore, Singapore","{'fileName': 'apollo.svg', 'hash': 'peWJyyZdPz..."
4,member,Provisioning,Automation & Configuration,aws-cloud-formation,AWS CloudFormation,AWS CloudFormation,<NA>,<NA>,<NA>,NaN,0,NaN,NaN,NaN,"[Consulting, DevOps, Information Services, Inf...","Seattle, Washington","{'fileName': 'aws-cloud-formation.svg', 'hash'..."


In [120]:
print(f'before nulls removed: {df.shape}')
df = df.dropna().copy()
print(f'after nulls removed: {df.shape}')

before nulls removed: (2276, 17)
after nulls removed: (226, 17)


In [121]:
categories = df['category'].drop_duplicates().tolist()
print(f'CATEGORIES\n{categories}\n')

subcategories = df['subcategory'].drop_duplicates().tolist()
print(f'SUBCATEGORIES\n{subcategories}\n')

org_names = df['org_name'].drop_duplicates().tolist()
print(f'ORG_NAMES\n{org_names}\n')


CATEGORIES
['Provisioning', 'Runtime', 'Orchestration & Management', 'App Definition and Development', 'Platform', 'Serverless', 'Observability and Analysis', 'Wasm']

SUBCATEGORIES
[' Automation & Configuration', ' Container Registry', ' Security & Compliance', ' Key Management', ' Cloud Native Storage', ' Container Runtime', ' Cloud Native Network', ' Scheduling & Orchestration', ' Coordination & Service Discovery', ' Remote Procedure Call', ' Service Proxy', ' API Gateway', ' Service Mesh', ' Database', ' Streaming & Messaging', ' Application Definition & Image Build', ' Continuous Integration & Delivery', ' Certified Kubernetes - Distribution', ' Certified Kubernetes - Hosted', ' Certified Kubernetes - Installer', 'Container Service', ' Framework', ' Installable Platform', ' Monitoring', ' Feature Flagging', ' Logging', ' Tracing', ' Chaos Engineering', ' Continuous Optimization', ' Runtimes', ' Orchestration & Management']

ORG_NAMES
['project-akri', 'cdk8s-team', 'cloud-custodian

In [122]:
df.head()

,relation,category,subcategory,id,name,flatName,repo,repo_name,org_name,contributorsCount,commitsThisYear,stars,github_data,extra,industries,headquarters,image_data
1,sandbox,Provisioning,Automation & Configuration,akri,Akri,Akri,project-akri/akri,akri,project-akri,34.0,197,996.0,"{'languages': [{'name': 'Rust', 'value': 10535...","{'accepted': '2021-09-14', 'annual_review_date...","[Cloud Computing, Cloud Infrastructure, Non Pr...","San Francisco, California","{'fileName': 'akri.svg', 'hash': '2nK42JQaM8qF..."
7,sandbox,Provisioning,Automation & Configuration,cdk-for-kubernetes-cdk8s,CDK for Kubernetes (CDK8s),CDK for Kubernetes (CDK8s),cdk8s-team/cdk8s,cdk8s,cdk8s-team,78.0,434,3818.0,"{'languages': [{'name': 'JavaScript', 'value':...","{'accepted': '2020-11-10', 'annual_review_url'...","[Cloud Computing, Cloud Infrastructure, Non Pr...","San Francisco, California","{'fileName': 'cdk-for-kubernetes-cdk8s.svg', '..."
10,incubating,Provisioning,Automation & Configuration,cloud-custodian,Cloud Custodian,Cloud Custodian,cloud-custodian/cloud-custodian,cloud-custodian,cloud-custodian,441.0,522,4951.0,"{'languages': [{'name': 'Python', 'value': 771...","{'accepted': '2020-06-25', 'dev_stats_url': 'h...","[Cloud Computing, Cloud Infrastructure, Non Pr...","San Francisco, California","{'fileName': 'cloud-custodian.svg', 'hash': '0..."
13,sandbox,Provisioning,Automation & Configuration,dev-stream,DevStream,DevStream,devstream-io/devstream,devstream,devstream-io,62.0,515,815.0,"{'languages': [{'name': 'Go', 'value': 31401, ...","{'accepted': '2022-06-17', 'dev_stats_url': 'h...","[Cloud Computing, Cloud Infrastructure, Non Pr...","San Francisco, California","{'fileName': 'dev-stream.svg', 'hash': 'R9wN8R..."
18,sandbox,Provisioning,Automation & Configuration,kcl,KCL,KCL,kcl-lang/kcl,kcl,kcl-lang,15.0,329,808.0,"{'languages': [{'name': 'Rust', 'value': 28654...","{'accepted': '2023-09-20', 'dev_stats_url': 'h...","[Cloud Computing, Cloud Infrastructure, Non Pr...","San Francisco, California","{'fileName': 'kcl.svg', 'hash': 'vzDONy6/3IQev..."


In [123]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 226 entries, 1 to 2242
Data columns (total 17 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   relation           226 non-null    object 
 1   category           226 non-null    object 
 2   subcategory        226 non-null    object 
 3   id                 226 non-null    object 
 4   name               226 non-null    object 
 5   flatName           226 non-null    object 
 6   repo               226 non-null    string 
 7   repo_name          226 non-null    string 
 8   org_name           226 non-null    string 
 9   contributorsCount  226 non-null    float64
 10  commitsThisYear    226 non-null    int64  
 11  stars              226 non-null    float64
 12  github_data        226 non-null    object 
 13  extra              226 non-null    object 
 14  industries         226 non-null    object 
 15  headquarters       226 non-null    object 
 16  image_data         226 non-nul

In [124]:
df.describe()

,contributorsCount,commitsThisYear,stars
count,226.000000,226.000000,226.000000
mean,218.530973,962.371681,5351.601770
std,428.580200,1707.273141,9788.969626
min,4.000000,0.000000,8.000000
25%,35.250000,126.250000,765.750000
50%,80.500000,399.000000,1959.500000
75%,245.500000,976.000000,5888.250000
max,4842.000000,12533.000000,101881.000000


In [125]:
df.index

Index([   1,    7,   10,   13,   18,   20,   21,   22,   29,   30,
       ...
        987,  996,  998, 1001, 1015, 2166, 2168, 2172, 2173, 2242], dtype='int64', length=226)

## Set Index 

In [126]:
# index --> ['relation', 'category', 'subcategory', 'id']

#safe_set_index(df, ['relation', 'category', 'subcategory', 'id'])
safe_set_index(df, ['category', 'subcategory', 'relation', 'id'])

df_numeric = df.select_dtypes(include=['int64', 'float64'])
df_numeric

	 Index changed from [None] --> ['category', 'subcategory', 'relation', 'id']


contributorsCount  commitsThisYear     stars
category                       subcategory                           relation   id                                                                               
App Definition and Development  Application Definition & Image Build False      gefyra                                            13.0              786     550.0
                                                                                mirrord                                           28.0              846    2949.0
                                                                     graduated  helm                                             766.0              407   24953.0
                                                                     incubating backstage                                       1629.0            12533   23285.0
                                                                                buildpacks                                       123.0              951    2139.0
                                                                                dapr                                             247.0              967   22320.0
                                                                                kube-vela                                        225.0              708    5598.0
                                                                                kube-virt                                        341.0             2588    4529.0
                                                                                operator-framework                               334.0              149    6706.0
                                                                     member     quarkus                                          984.0             7580   12421.0
                                                                     sandbox    artifact-hub                                      35.0              254    1364.0
                                                                                carvel                                            68.0               54    1477.0
                                                                                dev-space                                          6.0                0      99.0
                                                                                devfile                                           22.0               15     192.0
                                                                                ko                                                83.0              253    6328.0
                                                                                konveyor                                          14.0              137      18.0
                                                                                krator                                            24.0                0     133.0
                                                                                kudo                                              65.0                0    1127.0
                                                                                microcks                                          36.0              180     940.0
                                                                                nocalhost                                         50.0               36    1681.0
                                                                                porter                                            97.0              492    1053.0
                                                                                sealer                                            74.0              227    1921.0
                                                                                serverless-workflow                               41.0               40     620.0
                                                                   

In [127]:
# Aggregation at level 0
df_level0_relation = df_numeric.groupby(level=0).sum()  # or .mean(), .count(), etc.

# Aggregation at level 1
df_level1_category = df_numeric.groupby(level=1).sum()  # or .mean(), .count(), etc.

# Aggregation at level 2
df_level2_subcategory = df_numeric.groupby(level=2).sum()  # or .mean(), .count(), etc.

# Aggregation at level 3
# df_level3 = df.groupby(level=3).sum()  # or .mean(), .count(), etc.

In [128]:
print(f'Aggregations at df_level0_relation\n\n{df_level0_relation}\n')
print(f'Aggregations at df_level1_category\n\n{df_level1_category}\n')
print(f'Aggregations at df_level2_subcategory\n\n{df_level2_subcategory}\n')

Aggregations at df_level0_relation

                                contributorsCount  commitsThisYear     stars
category                                                                    
App Definition and Development            13186.0            77222  296517.0
Observability and Analysis                 5877.0            13426  155993.0
Orchestration & Management                14796.0            43105  357274.0
Platform                                    814.0             5681   26603.0
Provisioning                               7979.0            41827  185859.0
Runtime                                    4904.0            20183  121266.0
Serverless                                  864.0             2143   35530.0
Wasm                                        968.0            13909   30420.0

Aggregations at df_level1_category

                                       contributorsCount  commitsThisYear     stars
subcategory                                                              

In [129]:
# print(f'df.index.names: {df.index.names}\n\n')
# #print(f'df.index.levels: {df.index.levels}\n\n')

# for level in df.index.levels:
#     print(f'level: {level}\n')

## Generate Sunbursts and Treemaps for { Contributors Count, Commits this year, Stars }

### Helpers

In [130]:
import plotly.io as pio
import plotly.express as px
import plotly.graph_objects as go

In [131]:
# Light to Dark Transition:
# color_discrete_map={'sandbox': '#ADD8E6', 'incubating': '#87CEEB', 'graduated': '#000080'}
#
# sandbox: Light Blue (#ADD8E6)
# incubating: Medium Blue (#87CEEB)
# graduated: Dark Blue (#000080)

# Warm to Cool Transition:
# color_discrete_map={'sandbox': '#FFA500', 'incubating': '#FFD700', 'graduated': '#008000'}
#
# sandbox: Orange (#FFA500)
# incubating: Yellow (#FFD700)
# graduated: Green (#008000)

# Warm to Cool 2 Transition:
# color_discrete_map={'sandbox': '#FFD700', 'incubating': '#87CEEB', 'graduated': '#008000'}
#
# sandbox: Yellow (#FFD700)
# incubating: Medium Blue (#87CEEB)
# graduated: Green (#008000)

color_discrete_map_pastel = {
    "App Definition and Development": "#a2cffe",
    "Observability and Analysis": "#8efac1",
    "Orchestration & Management": "#fc9d9a",
    "Platform": "#c0eb75",
    "Provisioning": "#f2a2e8",
    "Runtime": "#fffe7a",
    "Serverless": "#d3d3d3"
}

color_discrete_map1 = {
    "App Definition and Development": "#264653",
    "Observability and Analysis": "#2a9d8f",
    "Orchestration & Management": "#e9c46a",
    "Platform": "#f4a261",
    "Provisioning": "#e76f51",
    "Runtime": "#6d6875",
    "Serverless": "#fca311"
}


color_discrete_map2 = {
    "App Definition and Development": "#003f5c",
    "Observability and Analysis": "#58508d",
    "Orchestration & Management": "#bc5090",
    "Platform": "#ff6361",
    "Provisioning": "#ffa600",
    "Runtime": "#2f4b7c",
    "Serverless": "#665191"
}
color_discrete_map3 = {
    "App Definition and Development": "#165aa7",
    "Observability and Analysis": "#cb495c",
    "Orchestration & Management": "#bb60d5",
    "Platform": "#f47915",
    "Provisioning": "#06ab54",
    "Runtime": "#002070",
    "Serverless": "#b27d12"
}

color_discrete_map4 = {
    "App Definition and Development": "#1f77b4",
    "Observability and Analysis": "#ff7f0e",
    "Orchestration & Management": "#2ca02c",
    "Platform": "#d62728",
    "Provisioning": "#9467bd",
    "Runtime": "#8c564b",
    "Serverless": "#e377c2"
}


In [132]:
def create_figure(plotly_func,
                  df,
                  values=None,
                  height: int = 1200,
                  width: int = 1200,
                  title: str = 'Missing Title',
                  path=['category', 'subcategory', 'relation', 'id'],
                  color='category',
                  color_discrete_map=color_discrete_map2,
                  branchvalues: str = None) -> go.Figure:
    
    fig = plotly_func(data_frame=df,
                      values=values,
                      height=height,
                      width=width,
                      title=title,
                      path=path,
                      color=color,
                      color_discrete_map=color_discrete_map,
                      branchvalues='total')
    return fig


def create_sunburst(df, **kwargs) -> go.Figure:
    return create_figure(px.sunburst, df, **kwargs)


def create_treemap(df, **kwargs) -> go.Figure:
    return create_figure(px.treemap, df, **kwargs)

### Create Figures (Sunbursts, Treemaps) 

In [133]:
# make index columns accessible for charting as normal columns
df_reset = df.reset_index()

figs = {}

sunb_contributorsCount = create_sunburst(df_reset, values='contributorsCount', title='sunburst: 🪴 Contributor Count (NOT UNIQUE ACROSS PROJECTS!) 🪴')
sunb_commitsThisYear   = create_sunburst(df_reset, values='commitsThisYear',   title='sunburst: 📄 Commits This Year 📄')
sunb_stars             = create_sunburst(df_reset, values='stars',             title='sunburst: ⭐ Stars ⭐')

tree_contributorsCount = create_treemap(df_reset, values='contributorsCount',  title='treemap: 🪴 Contributor Count (NOT UNIQUE ACROSS PROJECTS!) 🪴')
tree_commitsThisYear   = create_treemap(df_reset, values='commitsThisYear',    title='treemap: 📄 Commits This Year 📄')
tree_stars             = create_treemap(df_reset, values='stars',              title='treemap: ⭐ Stars ⭐')

figs['sunb_contributorsCount'] = sunb_contributorsCount
figs['sunb_commitsThisYear']   = sunb_commitsThisYear
figs['sunb_stars']             = sunb_stars

figs['tree_contributorsCount'] = tree_contributorsCount
figs['tree_commitsThisYear']   = tree_commitsThisYear
figs['tree_stars']             = tree_stars

### Save figures as .svg files

In [144]:
for key, fig in figs.items():
    file_name = f'{OUT_DIR}/fig_{key}.svg'
    fig.write_image(file_name, format='svg')

    # Emit raw markdown for image description
    markdown = f"![Image description]({file_name})"
    print(f"```{markdown}```")

```![Image description](generated/fig_sunb_contributorsCount.svg)```
```![Image description](generated/fig_sunb_commitsThisYear.svg)```
```![Image description](generated/fig_sunb_stars.svg)```
```![Image description](generated/fig_tree_contributorsCount.svg)```
```![Image description](generated/fig_tree_commitsThisYear.svg)```
```![Image description](generated/fig_tree_stars.svg)```


In [135]:
# save --> files so viewable in markdown cells
#
# ![Image description](path_to_image.png)

### Display Sunbursts

In [136]:
figs['sunb_stars'].show()

In [137]:
figs['sunb_contributorsCount'].show()

In [138]:
figs['sunb_commitsThisYear'].show()

### Display Treemaps

In [139]:
figs['tree_contributorsCount'].show()

In [140]:
figs['tree_commitsThisYear'].show()

In [141]:
figs['tree_contributorsCount'].show()

### images for github rendering

![Image description](generated/fig_sunb_contributorsCount.svg)
![Image description](generated/fig_sunb_commitsThisYear.svg)
![Image description](generated/fig_sunb_stars.svg)
![Image description](generated/fig_tree_contributorsCount.svg)
![Image description](generated/fig_tree_commitsThisYear.svg)
![Image description](generated/fig_tree_stars.svg)

## Generate Per TAG views.

In [145]:
print(df.info())
print(f'index.names: {df.index.names}')
df.head()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 226 entries, ('App Definition and Development', ' Application Definition & Image Build', False, 'gefyra') to ('Wasm', ' Runtimes', 'sandbox', 'wasm-edge-wasm')
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   name               226 non-null    object 
 1   flatName           226 non-null    object 
 2   repo               226 non-null    string 
 3   repo_name          226 non-null    string 
 4   org_name           226 non-null    string 
 5   contributorsCount  226 non-null    float64
 6   commitsThisYear    226 non-null    int64  
 7   stars              226 non-null    float64
 8   github_data        226 non-null    object 
 9   extra              226 non-null    object 
 10  industries         226 non-null    object 
 11  headquarters       226 non-null    object 
 12  image_data         226 non-null    object 
dtypes: float64(2), int64(1), object(

name    flatName                  repo  repo_name      org_name  contributorsCount  commitsThisYear    stars                                        github_data                                              extra                                         industries               headquarters                                         image_data
category                       subcategory                           relation   id                                                                                                                                                                                                                                                                                                                                                                   
App Definition and Development  Application Definition & Image Build False      gefyra          Gefyra      Gefyra       gefyrahq/gefyra     gefyra      gefyrahq               13.0              786    550.0  {'languages': [{'name': 'Python', 'value': 386...     {'slack_url': 'https://discord.gg/8NTPMVPaKy'}             [Marketing, Software, Web Development]            Munich, Germany  {'fileName': 'gefyra.svg', 'hash': 'vGElpZyBbf...
                                                                                mirrord        mirrord     mirrord  metalbear-co/mirrord    mirrord  metalbear-co               28.0              846   2949.0  {'languages': [{'name': 'Rust', 'value': 15028...  {'slack_url': 'https://discord.com/invite/pSKE...  [Developer Tools, Information Technology, Open...           Tel Aviv, Israel  {'fileName': 'mirrord.svg', 'hash': '866NcKcVN...
                                                                     graduated  helm              Helm        Helm             helm/helm       helm          helm              766.0              407  24953.0  {'languages': [{'name': 'Go', 'value': 1807584...  {'accepted': '2018-06-01', 'incubating': '2018...  [Cloud Computing, Cloud Infrastructure, Non Pr...  San Francisco, California  {'fileName': 'helm.svg', 'hash': 'KhA6o4K6XUSq...
                                                                     incubating backstage    Backstage   Backstage   backstage/backstage  backstage     backstage             1629.0            12533  23285.0  {'languages': [{'name': 'TypeScript', 'value':...  {'accepted': '2020-09-08', 'incubating': '2022...  [Cloud Computing, Cloud Infrastructure, Non Pr...  San Francisco, California  {'fileName': 'backstage.svg', 'hash': 'UDo/uyF...
                                                                                buildpacks  Buildpacks  Buildpacks       buildpacks/pack       pack    buildpacks              123.0              951   2139.0  {'languages': [{'name': 'Go', 'value': 2266863...  {'accepted': '2018-10-03', 'incubating': '2020...  [Cloud Computing, Cloud Infrastructure, Non Pr...  San Francisco, California  {'fileName': 'buildpacks.svg', 'hash': 'yLJvp/...

In [148]:
df.info()
safe_set_index(df, ['category', 'subcategory', 'relation', 'id'])


<class 'pandas.core.frame.DataFrame'>
MultiIndex: 226 entries, ('App Definition and Development', ' Application Definition & Image Build', False, 'gefyra') to ('Wasm', ' Runtimes', 'sandbox', 'wasm-edge-wasm')
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   name               226 non-null    object 
 1   flatName           226 non-null    object 
 2   repo               226 non-null    string 
 3   repo_name          226 non-null    string 
 4   org_name           226 non-null    string 
 5   contributorsCount  226 non-null    float64
 6   commitsThisYear    226 non-null    int64  
 7   stars              226 non-null    float64
 8   github_data        226 non-null    object 
 9   extra              226 non-null    object 
 10  industries         226 non-null    object 
 11  headquarters       226 non-null    object 
 12  image_data         226 non-null    object 
dtypes: float64(2), int64(1), object(

In [149]:
# debug - just Observability TAG Projects
#repos_by_relation = df.query("`category` == 'Observability and Analysis'")[['relation','repo', 'name']].copy()



repos_by_relation = df.copy()
repos_by_relation.groupby('relation')['repo'].agg(lambda x: list(x)).to_dict()

safe_set_index(repos_by_relation, idx_wanted=['relation', 'name'])
repos_by_relation.sort_index()

KeyError: "None of ['relation'] are in the columns"

In [ ]:
graduated_single_repos  = repos_by_relation.loc['graduated', :]['repo'].tolist()
incubating_single_repos = repos_by_relation.loc['incubating', :]['repo'].tolist()
sandbox_single_repos    = repos_by_relation.loc['sandbox', :]['repo'].tolist()

display(graduated_single_repos, incubating_single_repos, sandbox_single_repos)

## Fetch project release data from GitHub API

In [ ]:
import time
import os
import json
import pandas as pd
from typing import List
from datetime import datetime, timezone
from github import Github, GithubException

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 150)

def fetch_repo_data(token: str, 
                    repo_list: List[str], 
                    since: datetime=None, 
                    json_file: str=None, 
                    csv_file: str=None, 
                    state_file: str=None) -> pd.DataFrame:

    # Initialize DataFrame
    # df = pd.DataFrame(columns=[
    #     'repo_name', 'release_name', 'release_date', 
    #     'language', 'release_notes'
    # ])

    df = pd.DataFrame(columns=['repo_name', 'release_name', 'release_date', 'language'])

    # Initialize GitHub client
    g = Github(token)

    # Initialize loop state
    if state_file:
        try:
            with open(state_file, 'r') as f:
                state = json.load(f)
        except FileNotFoundError:
            state = {'i': 0, 'repos_done': []}
    else:
        state = {'i': 0, 'repos_done': []}

    # Loop over repositories
    while state['i'] < len(repo_list):
        repo_str = repo_list[state['i']]

        if repo_str in state['repos_done']:
            print(f"Skipping: {repo_str}")
            state['i'] += 1
            continue

        while True:
            try:
                repo = g.get_repo(repo_str)
                break
            except GithubException as e:
                if e.status == 404:
                    print(f"Repository {repo_str} not found")
                    break
                elif e.status == 429:
                    print(f"Rate limit exceeded, waiting for {e.headers['Retry-After']} seconds...")
                    time.sleep(int(e.headers['Retry-After']))
                else:
                    print(f"Error getting repository {repo_str}: {e}")
                    break

        if not repo:
            state['i'] += 1
            continue

        #
        # Get all releases
        #
        releases = repo.get_releases()
        language = repo.language

        for release in releases:
            if since is None or release.created_at >= since:
                df = pd.concat([df, pd.DataFrame({
                    'repo_name': [repo_str],
                    'release_name': [release.title],
                    'release_date': [str(release.published_at)],
                    'language': [language],
                    #'release_notes': [release.body]
                })])
                print(f"Added {release.published_at}, {repo_str}::{release.title}  ")

        # Save state
        if state_file:
            state['repos_done'].append(repo_str)
            with open(state_file, 'w') as f:
                json.dump(state, f, indent=4)

        state['i'] += 1

    #print (releases)
    
    # Save as CSV
    if csv_file:
        df.to_csv(csv_file, index=False)

    # Save as JSON
    if json_file:
        df.to_json(json_file, orient='records', lines=True)
    return df

In [ ]:
!mkdir -p out

In [ ]:
def fetch_one(token, since_date, level, repos):

    json_file=f'out/{level}-github-releases.json' 
    csv_file=f'out/{level}-github-releases.csv'
    state_file=f'out/.nukeme_state_file_{level}'
    
    print(f"Fetching {len(repos)} repositories for {level} projects")
    
    releases = fetch_repo_data( token, 
                                repos, 
                                since=since_date,
                                json_file=json_file,
                                csv_file=csv_file)

In [ ]:
token = os.environ['GITHUB_TOKEN']
since_date = datetime(2022, 11, 7, tzinfo=timezone.utc)

In [ ]:
fetch_one(token, since_date, 'cncf-graduated', graduated_single_repos)

In [ ]:
fetch_one(token, since_date, 'cncf-incubating', incubating_single_repos)

In [ ]:
fetch_one(token, since_date, 'cncf-sandbox', sandbox_single_repos)

## Visualize Releases

In [ ]:
%pip install ipympl

%matplotlib inline
%matplotlib widget

In [ ]:
import plotly.express as px
import matplotlib.pyplot as plt

# def plot_releases_timeline(releases: pd.DataFrame):
#     fig = px.timeline(releases, x_start="release_date", x_end="release_date", y="repo_name", color="language", title="GitHub Releases Timeline")
#     fig.update_yaxes(autorange="reversed")
#     fig.show()

# def plot_releases_scatter_simple(releases: pd.DataFrame):
#     # Filter releases by year
#     releases_2023 = releases[releases['release_date'].dt.year == 2023]

#     # Create scatter plot
#     fig = px.scatter(releases_2023, x="release_date", y="repo_name", color="language")
#     fig.update_yaxes(autorange="reversed")
#     fig.show()

def plot_releases_scatter(releases: pd.DataFrame, title: str=None):
    # # Filter releases by year
    # releases_2023 = releases[releases['release_date'].dt.year == 2023]

    if title is None:
        title = "GitHub Releases Timeline"

    # Group releases by organization
    releases['organization'] = releases['repo_name'].apply(lambda x: x.split('/')[0])

    # Create scatter plot
    fig = px.scatter(releases, x="release_date", y="repo_name", color="organization", symbol="language", title="Project Releases")
    fig.update_yaxes(autorange="reversed")
    
    fig.update_layout(showlegend=True,
                      autosize=True,
                      width=1000)
                    #   height=2500,
                    #   )
    fig.show()

In [ ]:
import pandas as pd
import os

def json_to_csv(json_file_path: str) -> None:
    """
    Load a JSON file into a pandas DataFrame and save it as a CSV file with the same name.
    """
    df = pd.read_json(json_file_path, lines=True)
    
    csv_file_path = os.path.splitext(json_file_path)[0] + '.csv'
    df.to_csv(csv_file_path, index=False)

In [ ]:
for level in ['graduated', 'incubating', 'sandbox']:
    json_to_csv(f'out/cncf-{level}-github-releases.json')

In [ ]:
def plot_releases_from_csv(csv_file: str, title: str) -> None:
    csv_file =f'out/cncf-{level}-github-releases.csv'
    if os.path.exists(csv_file):
        df_releases = pd.read_csv(csv_file)
        df_releases.release_date = pd.to_datetime(df_releases.release_date)
        
        plot_releases_scatter(df_releases, title)
    else:
        print(f"CSV file {csv_file} not found")

In [ ]:
import pandas as pd
import os

for level in ['graduated', 'incubating', 'sandbox']:
    plot_releases_from_csv(f'out/cncf-{level}-github-releases.csv', f'Releases: {level}')


In [ ]:
import pandas as pd
import os


# plot_releases_scatter(f'out/cncf-all-github-releases.csv')



In [ ]:
releases_by_repo = releases[['repo_name', 'release_date']].groupby('repo_name').count()
releases_by_repo.to_csv('cncf_releases_by_repo.csv')